In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def get_weather(lon, lat):
  print("call an api to get weather info")

function = {
  "name": "get_weather",
  "description": "function that takes longitude and latitude to find the weather of a place",
  "parameters": {
    "type": "object",
    "properties": {
      "lon": {"type": "string", "description": "The longitude coordinate"},
      "lat": {"type": "string", "description": "The latitude coordinate"},
    },
  },
  "required": ["lon", "lat"],
}

llm = ChatOpenAI(temperature=0.1).bind(function_call="auto", functions=[function])
prompt = PromptTemplate.from_template("What is the weather in {city}")

chain = prompt | llm

response = chain.invoke({
  "city": "Seoul"
})

response = response.additional_kwargs["function_call"]["arguments"]


In [7]:
import json

response = json.loads(response)

get_weather(response["lon"], response["lon"])

call an api to get weather info


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "Seoul"})


response = response.additional_kwargs["function_call"]["arguments"]

response

'{\n  "questions": [\n    {\n      "question": "What is the capital city of South Korea?",\n      "answers": [\n        {\n          "answer": "Seoul",\n          "correct": true\n        },\n        {\n          "answer": "Busan",\n          "correct": false\n        },\n        {\n          "answer": "Incheon",\n          "correct": false\n        },\n        {\n          "answer": "Daegu",\n          "correct": false\n        }\n      ]\n    },\n    {\n      "question": "Which river flows through Seoul?",\n      "answers": [\n        {\n          "answer": "Han River",\n          "correct": true\n        },\n        {\n          "answer": "Nakdong River",\n          "correct": false\n        },\n        {\n          "answer": "Geum River",\n          "correct": false\n        },\n        {\n          "answer": "Yeongsan River",\n          "correct": false\n        }\n      ]\n    },\n    {\n      "question": "What is the tallest building in Seoul?",\n      "answers": [\n        {\n 

In [9]:
import json
json.loads(response)

{'questions': [{'question': 'What is the capital city of South Korea?',
   'answers': [{'answer': 'Seoul', 'correct': True},
    {'answer': 'Busan', 'correct': False},
    {'answer': 'Incheon', 'correct': False},
    {'answer': 'Daegu', 'correct': False}]},
  {'question': 'Which river flows through Seoul?',
   'answers': [{'answer': 'Han River', 'correct': True},
    {'answer': 'Nakdong River', 'correct': False},
    {'answer': 'Geum River', 'correct': False},
    {'answer': 'Yeongsan River', 'correct': False}]},
  {'question': 'What is the tallest building in Seoul?',
   'answers': [{'answer': 'Lotte World Tower', 'correct': True},
    {'answer': 'N Seoul Tower', 'correct': False},
    {'answer': '63 Building', 'correct': False},
    {'answer': 'Seoul Sky', 'correct': False}]},
  {'question': 'Which traditional market is famous in Seoul?',
   'answers': [{'answer': 'Namdaemun Market', 'correct': True},
    {'answer': 'Gwangjang Market', 'correct': False},
    {'answer': 'Dongdaemun Ma